In [11]:
import glob
import cv2
import numpy as np
import os
from tqdm import tqdm


# CUSTOM IMPLEMENTATION

### SIFT-Based Feature Matching

The function `detectFeaturesAndMatch` aims to establish a correspondence between keypoints in two images using Scale-Invariant Feature Transform (SIFT) and Brute-Force Matching. SIFT is an image descriptor for image-based matching and object recognition that is invariant to changes in scale, rotation, and partially invariant to affine distortions and changes in illumination. Mathematically, SIFT descriptors are high-dimensional vectors extracted from a sub-region around each keypoint and are highly distinctive. The formula to calculate the SIFT descriptor involves the gradient magnitude and direction calculated in the keypoint's neighborhood.

$
\text{Descriptor} = f\left(\frac{\partial I(x,y)}{\partial x}, \frac{\partial I(x,y)}{\partial y}\right)
$

After SIFT feature extraction, the next step is matching the descriptors from two images. The function utilizes Brute-Force Matcher with L2 norm for this purpose. The Brute-Force matcher takes the descriptor of a feature in the first set and matches it with all other features in the second set using a distance calculation, often the Euclidean distance:

$
\text{Distance} = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}
$

The matches are then sorted by this distance, providing a set of best matches. This approach allows for identifying correspondences between two images, which can be useful in tasks like image stitching, object recognition, and more.


In [12]:
def detectFeaturesAndMatch(image1, image2, maxNumOfFeatures=30):
    '''
    Takes two images as input and returns a set of correspondences between the two images, matched using SIFT features.
    The matches are sorted from best to worst based on the Euclidean (L2) distance between feature descriptors.
    '''

    # Initialize the SIFT feature detector and descriptor
    siftDetector = cv2.SIFT_create()

    # Detect keypoints and compute descriptors for both images
    keypoints1, descriptors1 = siftDetector.detectAndCompute(image1, None)
    keypoints2, descriptors2 = siftDetector.detectAndCompute(image2, None)

    # Initialize BFMatcher with L2 norm for matching SIFT descriptors
    bruteForceMatcher = cv2.BFMatcher(cv2.NORM_L2)

    # Perform the matching between descriptors of two images
    rawMatches = bruteForceMatcher.match(descriptors1, descriptors2)

    # Sort the matches based on their distance attributes
    sortedMatches = sorted(rawMatches, key=lambda x: x.distance)

    # Initialize an empty list to store corresponding points between two images
    featureCorrespondences = []
    for match in sortedMatches:
        featureCorrespondences.append((keypoints1[match.queryIdx].pt, keypoints2[match.trainIdx].pt))

    print(f'Total number of matches: {len(featureCorrespondences)}')

    # Prepare source and destination points for homography
    sourcePoints = np.float32([point_pair[0] for point_pair in featureCorrespondences[:maxNumOfFeatures]]).reshape(-1, 1, 2)
    destinationPoints = np.float32([point_pair[1] for point_pair in featureCorrespondences[:maxNumOfFeatures]]).reshape(-1, 1, 2)

    return np.array(featureCorrespondences[:maxNumOfFeatures]), sourcePoints, destinationPoints

### Homography Estimation via Direct Linear Transformation (DLT) and RANSAC

The first function, `calculateHomography`, uses Direct Linear Transformation (DLT) to calculate the homography matrix, $H$, from a set of point correspondences. DLT works by transforming the problem into a system of linear equations. The core of DLT involves solving the equation $Ah = 0$, where $A$ is constructed using the coordinates of point correspondences, and $h$ is a vector that contains the flattened homography matrix. Singular Value Decomposition (SVD) is then used to solve this equation system.

$
A = \begin{bmatrix}
x' & y' & 1 & 0 & 0 & 0 & -x'x & -x'y & -x' \\
0 & 0 & 0 & x' & y' & 1 & -y'x & -y'y & -y' \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots
\end{bmatrix}
$
$
h = [h_{11}, h_{12}, h_{13}, h_{21}, h_{22}, h_{23}, h_{31}, h_{32}, h_{33}]^T
$

The second function, `getBestHomographyRANSAC`, incorporates the RANSAC (Random Sample Consensus) algorithm to robustly estimate the homography matrix. RANSAC aims to find the model parameters that generate the maximum number of inliers, thereby mitigating the impact of outliers. In each iteration, a random subset of point correspondences is chosen to estimate the homography matrix. Then, other points are tested against this estimated homography to find inliers based on a predefined threshold. The best estimated homography is updated if a trial results in more inliers than any previous iteration.

$
\text{Error} = \lVert \text{dst\_point} - \text{estimated\_dst} \rVert_2
$

Here, $\lVert \cdot \rVert_2$ denotes the L2 norm, and the error is checked against a threshold to classify inliers and outliers.


In [13]:
def calculateHomography(point_correspondences):
    '''
    Calculate the homography matrix using the Direct Linear Transformation (DLT) algorithm.
    Takes in a set of point correspondences and returns the 3x3 homography matrix.

    Parameters:
        point_correspondences (list): List of matched points between two images. Each element should be a tuple (src_point, dst_point).

    Returns:
        np.ndarray: The 3x3 homography matrix.
    '''
  
    # Initialize an empty list to construct matrix A, which will be used to solve for the homography
    matrix_A = []

    for correspondence in point_correspondences:
        # Extract source and destination coordinates from each correspondence
        src_point, dst_point = correspondence
        src_x, src_y = src_point[0], src_point[1]
        dst_x, dst_y = dst_point[0], dst_point[1]

        # Append the equations to matrix A, following the DLT algorithm
        matrix_A.append([src_x, src_y, 1, 0, 0, 0, -dst_x * src_x, -dst_x * src_y, -dst_x])
        matrix_A.append([0, 0, 0, src_x, src_y, 1, -dst_y * src_x, -dst_y * src_y, -dst_y])

    # Convert the list to a NumPy array for numerical computations
    matrix_A = np.asarray(matrix_A)

    # Perform Singular Value Decomposition (SVD) to solve for the homography matrix
    U, S, Vh = np.linalg.svd(matrix_A)

    # Extract the last row of Vh and normalize it to get the homography elements
    homography_elements = Vh[-1, :] / Vh[-1, -1]

    # Reshape to form the 3x3 homography matrix
    homography_matrix = homography_elements.reshape(3, 3)

    return homography_matrix  # Return the computed 3x3 homography matrix


In [14]:
def getBestHomographyRANSAC(correspondences, trials=1000, threshold=10, num_samples=4):
    '''
    Estimate the homography matrix using the RANSAC algorithm.
    
    Parameters:
        correspondences (list): List of point correspondences between two images.
        trials (int): Number of RANSAC trials.
        threshold (float): RANSAC threshold for determining inliers.
        num_samples (int): Number of random samples to choose for each trial.
        
    Returns:
        np.ndarray: The estimated 3x3 homography matrix.
        list: The random sample of point correspondences that led to the estimated homography.
    '''

    # Initialize variables to keep track of the best homography and inliers
    best_homography = None
    max_inliers = []
    
    # Iterate through RANSAC trials
    for i in tqdm(range(trials)):
        # Randomly select 'num_samples' correspondences
        selected_indices = np.random.choice(len(correspondences), num_samples, replace=False)
        random_sample = correspondences[selected_indices]
        
        # Compute the homography using the random sample
        estimated_homography = calculateHomography(random_sample)
        
        # Initialize list to keep track of inliers for this iteration
        current_inliers = []
        
        for correspondence in correspondences:
            # Prepare source and destination points
            src_point = np.append(correspondence[0], 1)
            dst_point = np.append(correspondence[1], 1)
            
            # Estimate the destination point based on the current homography
            estimated_dst = np.dot(estimated_homography, src_point)
            estimated_dst /= estimated_dst[-1]
            
            # Calculate the error (Euclidean distance)
            error = np.linalg.norm(dst_point - estimated_dst)
            
            # Check if this correspondence is an inlier
            if error < threshold:
                current_inliers.append(correspondence)
        
        # Update if this trial gives more inliers than before
        if len(current_inliers) > len(max_inliers):
            max_inliers = current_inliers
            best_homography = estimated_homography

    print(f"Max inliers = {len(max_inliers)}")
    return best_homography, random_sample

### Computing the Bounding Box of a Warped Image

The function `computeBoundingBoxOfWarpedImage` calculates the bounding box for an image after it undergoes a homographic transformation, defined by a given 3x3 homography matrix $H$. The bounding box is essential for stitching or overlaying the transformed image onto another canvas.

To compute the bounding box, we first define the four corner points of the original image in homogeneous coordinates: \([x, y, 1]^T\). The four corners are the top-left \((0, 0)\), top-right \((\text{img\_width}-1, 0)\), bottom-left \((0, \text{img\_height}-1)\), and bottom-right \((\text{img\_width}-1, \text{img\_height}-1)\) points. These points are encapsulated into a 3x4 matrix $P$.

$
P = \begin{bmatrix}
0 & \text{img\_width}-1 & 0 & \text{img\_width}-1 \\
0 & 0 & \text{img\_height}-1 & \text{img\_height}-1 \\
1 & 1 & 1 & 1
\end{bmatrix}
$

We then apply the homography matrix $H$ to $P$ to find the transformed corner points:

$
\text{Transformed\_corners} = H \times P
$

After transforming, the coordinates are normalized to inhomogeneous form by dividing each coordinate by its $w$-component. Finally, the bounding box is determined by finding the minimum and maximum $x$ and $y$ coordinates among the transformed corners.

$
(x_{\text{min}}, x_{\text{max}}, y_{\text{min}}, y_{\text{max}}) = (\min(x), \max(x), \min(y), \max(y))
$

This bounding box is crucial for understanding the spatial requirements when integrating the transformed image with other images or backgrounds.


In [15]:
def computeBoundingBoxOfWarpedImage(homography_matrix, img_width, img_height):
    """
    Compute the bounding box of an image after it has been transformed by a given homography matrix.

    Args:
        homography_matrix (np.ndarray): 3x3 matrix used for the homographic transformation.
        img_width (int): Width of the original image.
        img_height (int): Height of the original image.

    Returns:
        tuple: Coordinates of the bounding box as (x_min, x_max, y_min, y_max).
    """
    
    # Define corners of the original image [Top-left, Top-right, Bottom-left, Bottom-right]
    # Each column is a corner point in homogeneous coordinates (x, y, 1)
    original_corners = np.array([[0, img_width - 1, 0, img_width - 1], 
                                 [0, 0, img_height - 1, img_height - 1], 
                                 [1, 1, 1, 1]])
    
    # Apply the homography to the original corners
    transformed_corners = np.dot(homography_matrix, original_corners)
    
    # Convert to inhomogeneous coordinates by dividing by the third (w) component
    transformed_corners /= transformed_corners[2, :]
    
    # Find the minimum and maximum x and y coordinates of the transformed corners
    x_min = np.min(transformed_corners[0])
    x_max = np.max(transformed_corners[0])
    y_min = np.min(transformed_corners[1])
    y_max = np.max(transformed_corners[1])
    
    return int(x_min), int(x_max), int(y_min), int(y_max)

# `warpAndPlaceSourceImage` Function Analysis

The function `warpAndPlaceSourceImage` performs the task of warping a source image based on a given 3x3 homography matrix $H$ and places it onto a destination image. The function offers flexibility by allowing both forward and inverse mapping techniques and also allows for a positional offset.

## Forward Mapping Method

In the forward mapping method, the homography matrix $H$ is directly applied to each coordinate of the source image. This approach is computationally expedient but can lead to holes or gaps in the destination image due to mapping conflicts (multiple source pixels mapping to the same destination pixel) or an absence of mapping (some destination pixels receiving no mapping).

### Steps:

1. **Generate Indices**: Create a grid of `(x, y)` coordinates that span the source image and convert these coordinates to their homogeneous form:

    $
    \text{Homogeneous Coords} = \begin{pmatrix} x \\ y \\ 1 \end{pmatrix}
    $

2. **Apply Homography**: Perform the dot product between $H$ and the homogeneous coordinates to get the transformed coordinates:

    $
    \text{Transformed Coords} = H \times \text{Homogeneous Coords}
    $
  
3. **Normalize**: Convert to inhomogeneous coordinates by dividing by the $w$-component.
  
4. **Place onto Destination**: The transformed coordinates are directly indexed into the destination image, and any offset is added at this stage.

## Inverse Mapping Method

In the inverse mapping method, each pixel in the destination image is traced back to its corresponding pixel in the source image. This method is computationally intensive but it eliminates the issue of holes or gaps by ensuring every destination pixel is adequately mapped.

### Steps:

1. **Compute Bounding Box**: First, use `computeBoundingBoxOfWarpedImage` to find the bounding box that the transformed source image will occupy on the destination image.

2. **Generate Indices**: Create a grid of `(x, y)` coordinates that reside within this bounding box in the destination image. Convert these to homogeneous coordinates.
  
3. **Apply Inverse Homography**: Multiply the homogeneous coordinates by $H^{-1}$ to find the corresponding coordinates in the source image:

    $
    \text{Transformed Coords} = H^{-1} \times \text{Homogeneous Coords}
    $

4. **Normalize and Filter**: Convert the coordinates back to their inhomogeneous form and perform boundary checks to ensure the coordinates lie within the dimensions of the source image.

5. **Map to Source**: Use the transformed coordinates to fetch the relevant pixels from the source image and map them onto the destination image. Any positional offset is added here.

This technique offers a more accurate mapping, making it more suitable for tasks that require a high level of precision such as image stitching or complex scene overlays.


In [16]:
def warpAndPlaceSourceImage(source_img, homography_matrix, dest_img, use_forward_mapping=False, offset=(0, 0)):
    """
    Warps the source image according to a given homography matrix and places it onto the destination image.

    Args:
        source_img (np.ndarray): The source image array to be warped.
        homography_matrix (np.ndarray): The 3x3 matrix governing the homographic transformation.
        dest_img (np.ndarray): The destination image array where the warped source image will be placed.
        use_forward_mapping (bool): Determines whether to use forward or inverse mapping for the transformation.
        offset (tuple): Offsets for placing the warped image onto the destination image in (x, y) coordinates.

    Returns:
        None: The destination image is modified in place.
    """

    # Extract the dimensions of the source image
    height, width, _ = source_img.shape
    
    # Compute the inverse of the homography matrix for inverse mapping
    homography_inv = np.linalg.inv(homography_matrix)

    if use_forward_mapping:
        # FORWARD MAPPING

        # Generate indices for source image coordinates
        coords = np.indices((width, height)).reshape(2, -1)
        homogeneous_coords = np.vstack((coords, np.ones(coords.shape[1])))
        
        # Apply the homography transformation
        transformed_coords = np.dot(homography_matrix, homogeneous_coords)
        transformed_coords /= transformed_coords[2, :]
        
        # Extract the integer coordinates of the output
        x_output, y_output = transformed_coords.astype(np.int32)[:2, :]
        
        # Place the transformed source image onto the destination image
        dest_img[y_output + offset[1], x_output + offset[0]] = source_img[coords[1], coords[0]]

    else:
        # INVERSE MAPPING
        
        # Compute bounding box of the warped image
        x_min, x_max, y_min, y_max = computeBoundingBoxOfWarpedImage(homography_matrix, width, height)
        
        # Generate indices for destination image coordinates within bounding box
        coords = np.indices((x_max - x_min, y_max - y_min)).reshape(2, -1)
        coords += np.array([[x_min], [y_min]])
        homogeneous_coords = np.vstack((coords, np.ones(coords.shape[1])))
        
        # Apply the inverse homography transformation
        transformed_coords = np.dot(homography_inv, homogeneous_coords)
        transformed_coords /= transformed_coords[2, :]
        
        # Extract the integer coordinates of the input
        x_input, y_input = transformed_coords.astype(np.int32)[:2, :]
        
        # Perform boundary check to make sure coordinates are within source image dimensions
        valid_indices = np.where((x_input >= 0) & (x_input < width) & (y_input >= 0) & (y_input < height))

        # Map the inverse-transformed points to the source image and place onto the destination image
        dest_img[coords[1, valid_indices] + offset[1], coords[0, valid_indices] + offset[0]] = source_img[y_input[valid_indices], x_input[valid_indices]]

# ImageBlenderWithPyramids Class Overview

The `ImageBlenderWithPyramids` class specializes in image blending using Gaussian and Laplacian pyramids. It is tailored for the 'STRAIGHTCUT' blending strategy. Here, we dissect the intricacies of each of its methods and their computational significance.

## Constructor: 

This initializes the class with the depth of the pyramids. Higher depth allows for more seamless blending but can be computationally intensive.

## Gaussian Pyramid: 

Generates a Gaussian pyramid for a given image. The pyramid is built through recursive down-sampling, primarily used for multi-scale image analysis.

## Laplacian Pyramid:

Generates a Laplacian pyramid, capturing the "details" or high-frequency components of the image. It's constructed from the Gaussian pyramid.

## Blended Pyramid: 

Blends two Laplacian pyramids using a Gaussian pyramid as the mask. It performs a weighted sum of the two pyramids, element-wise.

## Image Reconstruction:

Given a Laplacian pyramid, this function reconstructs the original image by starting from the smallest (last) image in the pyramid and progressively adding detail back into the image.

## Mask Generation: 

Generates a binary mask based on the non-zero regions of an image, which is instrumental in blending.

## Image Blending: 

The key function that ties everything together. It blends two images using Laplacian pyramids and a Gaussian mask. Masks are generated to identify overlapping regions and 'STRAIGHTCUT' is employed for blending.

## Computational Complexity

- Gaussian and Laplacian pyramid generation: $O(N \log N)$ for $N$ pixels.
- Blending and reconstruction: $O(N)$.




In [17]:
class ImageBlenderWithPyramids():
    '''
    Class for blending images using Gaussian and Laplacian pyramids.
    It is specialized for the 'STRAIGHTCUT' blending strategy.
    '''

    def __init__(self, pyramid_depth=6):
        '''
        Initialize with the depth of the pyramids.
        '''
        self.pyramid_depth = pyramid_depth

    def getGaussianPyramid(self, image):
        '''
        Generate a Gaussian pyramid for a given image.
        '''
        pyramid = [image]
        for _ in range(self.pyramid_depth - 1):
            image = cv2.pyrDown(image)
            pyramid.append(image)
        return pyramid

    def getLaplacianPyramid(self, image):
        '''
        Generate a Laplacian pyramid for a given image.
        '''
        pyramid = []
        for _ in range(self.pyramid_depth - 1):
            next_level_image = cv2.pyrDown(image)
            upsampled_image = cv2.pyrUp(next_level_image, dstsize=(image.shape[1], image.shape[0]))
            pyramid.append(image.astype(float) - upsampled_image.astype(float))
            image = next_level_image
        pyramid.append(image)
        return pyramid

    def getBlendingPyramid(self, laplacian_a, laplacian_b, gaussian_mask_pyramid):
        '''
        Create a blended Laplacian pyramid using two Laplacian pyramids and a Gaussian pyramid as mask.
        '''
        blended_pyramid = []
        for i, mask in enumerate(gaussian_mask_pyramid):
            triplet_mask = cv2.merge((mask, mask, mask))
            blended_pyramid.append(laplacian_a[i] * triplet_mask + laplacian_b[i] * (1 - triplet_mask))
        return blended_pyramid

    def reconstructFromPyramid(self, laplacian_pyramid):
        '''
        Reconstruct an image from its Laplacian pyramid.
        '''
        reconstructed_image = laplacian_pyramid[-1]
        for laplacian_level in reversed(laplacian_pyramid[:-1]):
            reconstructed_image = cv2.pyrUp(reconstructed_image, dstsize=laplacian_level.shape[:2][::-1]).astype(float) + laplacian_level.astype(float)
        return reconstructed_image

    def generateMaskFromImage(self, image):
        '''
        Generate a mask based on the non-zero regions of the image.
        '''
        mask = np.all(image != 0, axis=2)
        mask_image = np.zeros(image.shape[:2], dtype=float)
        mask_image[mask] = 1.0
        return mask_image

    def blendImages(self, image1, image2):
        '''
        Blend two images using Laplacian pyramids and a Gaussian mask.
        '''
        laplacian1 = self.getLaplacianPyramid(image1)
        laplacian2 = self.getLaplacianPyramid(image2)

        mask1 = self.generateMaskFromImage(image1).astype(np.bool_)
        mask2 = self.generateMaskFromImage(image2).astype(np.bool_)
        
        overlap_region = mask1 & mask2

        y_coords, x_coords = np.where(overlap_region)
        min_x, max_x = np.min(x_coords), np.max(x_coords)

        final_mask = np.zeros(image1.shape[:2])
        final_mask[:, :(min_x + max_x)//2] = 1.0

        gaussian_mask_pyramid = self.getGaussianPyramid(final_mask)

        blended_pyramid = self.getBlendingPyramid(laplacian1, laplacian2, gaussian_mask_pyramid)

        blended_image = self.reconstructFromPyramid(blended_pyramid)

        return blended_image, mask1, mask2

In [18]:
def stitch_and_save_images(src_idx, dest_idx, prev_homography):
    '''
    For a given pair of image indices, computes the best homography 
    matrix and saves the warped images to disk.

    Args:
        src_idx: Index of the source image in imagePaths list.
        dest_idx: Index of the destination image in imagePaths list.
        prev_homography: Previous cumulative homography matrix.

    Returns:
        new_cumulative_homography: Updated cumulative homography matrix.
    '''
    
    # Initialize the destination warped image with zeros
    warped_image = np.zeros((3000, 6000, 3), dtype=np.uint8)

    # Read source and destination images
    src_img = cv2.imread(imagePaths[src_idx])
    dest_img = cv2.imread(imagePaths[dest_idx])

    print(f'Original image size = {src_img.shape}')

    # Resize images for performance and compatibility
    resized_src_img = cv2.resize(src_img, shape)
    resized_dest_img = cv2.resize(dest_img, shape)

    # Detect features and perform matching between images
    matches, src_pts, dest_pts = detectFeaturesAndMatch(resized_dest_img, resized_src_img)

    # Compute the best homography using RANSAC
    best_homography, _ = getBestHomographyRANSAC(matches, trials=trials, threshold=threshold)

    # Update the cumulative homography matrix
    new_cumulative_homography = np.dot(prev_homography, best_homography)

    # Transform the source image and place it in the warped image
    warpAndPlaceSourceImage(resized_dest_img, new_cumulative_homography, dest_img=warped_image, offset=offset)

    # Save the warped image to disk
    cv2.imwrite(f'outputs/scene{imageSet}/custom/warped_{dest_idx}.png', warped_image)

    return new_cumulative_homography


In [24]:
import os
print(os.getcwd())


PermissionError: [Errno 1] Operation not permitted

In [2]:
import cv2
import numpy as np
import glob
import os

print("STARTING WITH CUSTOM IMPLEMENTATION")

for imageSet in range(1,7):  
    print(f"STARTING WITH IMAGESET - {imageSet}")
    imagePaths = sorted(glob.glob(f'dataset/scene{imageSet}/*'))
    print(imagePaths)
    shape = (600, 400)
    mid = len(imagePaths)//2
    threshold = 2
    trials = 3000
    offset = [2300, 800]
    if(imageSet in [1,2,3]):
        prevH = np.eye(3)
        prevH = stitch_and_save_images(2, 1, prevH)
        prevH = stitch_and_save_images(1, 0, prevH)

        prevH = np.eye(3)
        prevH = stitch_and_save_images(2, 2, prevH)
        prevH = stitch_and_save_images(2, 3, prevH)

        print("warping complete for set", imageSet)

        # WARPING COMPLETE. BLENDING START
        print("We are starting with blending")
        b = ImageBlenderWithPyramids()
        finalImg = cv2.imread(f'outputs/scene{imageSet}/custom/warped_0.png')

        for index in range(1, 4):
            print('blending', index)
            img2 = cv2.imread(f'outputs/scene{imageSet}/custom/warped_{index}.png')
            finalImg, mask1truth, mask2truth = b.blendImages(finalImg, img2)
            mask1truth = mask1truth + mask2truth
        
        cv2.imwrite(f'outputs/scene{imageSet}/custom/blended_image.png', finalImg)
    else:
        prevH = np.eye(3)
        prevH = stitch_and_save_images(1, 0, prevH)
        prevH = stitch_and_save_images(1, 1, prevH)

        print("warping complete for set", imageSet)

        # WARPING COMPLETE. BLENDING START
        print("We are starting with blending")
        b = ImageBlenderWithPyramids()
        finalImg = cv2.imread(f'outputs/scene{imageSet}/custom/warped_0.png')

        for index in range(1, 2):
            print('blending', index)
            img2 = cv2.imread(f'outputs/scene{imageSet}/custom/warped_{index}.png')
            finalImg, mask1truth, mask2truth = b.blendImages(finalImg, img2)
            mask1truth = mask1truth + mask2truth
        
        cv2.imwrite(f'outputs/scene{imageSet}/custom/blended_image.png', finalImg)


STARTING WITH CUSTOM IMPLEMENTATION
STARTING WITH IMAGESET - 1
['dataset/scene1/I11.JPG', 'dataset/scene1/I12.JPG', 'dataset/scene1/I13.JPG', 'dataset/scene1/I14.JPG']


NameError: name 'stitch_and_save_images' is not defined

# OPENCV IMPLEMENTATION

In [ ]:
def Homography_opencv(correspondences, trials=1000, threshold=10, num_samples=4):
        srcPoints = np.float32([point[0] for point in correspondences])
        dstPoints = np.float32([point[1] for point in correspondences])
        H, mask = cv2.findHomography(srcPoints, dstPoints, method=cv2.RANSAC, ransacReprojThreshold=threshold)
        return H, mask

In [ ]:
def stitch_and_save_images_opencv(src_idx, dest_idx, prev_homography):
    '''
    For a given pair of image indices, computes the best homography 
    matrix and saves the warped images to disk.

    Args:
        src_idx: Index of the source image in imagePaths list.
        dest_idx: Index of the destination image in imagePaths list.
        prev_homography: Previous cumulative homography matrix.

    Returns:
        new_cumulative_homography: Updated cumulative homography matrix.
    '''
    
    # Initialize the destination warped image with zeros
    warped_image = np.zeros((4192, 8192, 3), dtype=np.uint8)

    # Read source and destination images
    src_img = cv2.imread(imagePaths[src_idx])
    dest_img = cv2.imread(imagePaths[dest_idx])

    print(f'Original image size = {src_img.shape}')

    # Resize images for performance and compatibility
    resized_src_img = cv2.resize(src_img, shape)
    resized_dest_img = cv2.resize(dest_img, shape)

    # Detect features and perform matching between images
    matches, src_pts, dest_pts = detectFeaturesAndMatch(resized_dest_img, resized_src_img)

    # Compute the best homography using RANSAC
    best_homography, _ = Homography_opencv(matches, trials=trials, threshold=threshold)

    # Update the cumulative homography matrix
    new_cumulative_homography = np.dot(prev_homography, best_homography)

    # Transform the source image and place it in the warped image
    warpAndPlaceSourceImage(resized_dest_img, new_cumulative_homography, dest_img=warped_image, offset=offset)

    # Save the warped image to disk
    cv2.imwrite(f'outputs/scene{imageSet}/opencv/warped_{dest_idx}.png', warped_image)

    return new_cumulative_homography


In [ ]:
import cv2
import numpy as np
import glob
import os

print("STARTING WITH OPENCV IMPLEMENTATION")

for imageSet in range(1,7):  
    print(f"STARTING WITH IMAGESET - {imageSet}")
    imagePaths = sorted(glob.glob(f'dataset/scene{imageSet}/*'))
    shape = (600, 400)
    mid = len(imagePaths)//2
    threshold = 2
    trials = 5000
    offset = [2300, 800]
    if(imageSet in [1,2,3]):
        prevH = np.eye(3)
        prevH = stitch_and_save_images_opencv(2, 1, prevH)
        prevH = stitch_and_save_images_opencv(1, 0, prevH)

        prevH = np.eye(3)
        prevH = stitch_and_save_images_opencv(2, 2, prevH)
        prevH = stitch_and_save_images_opencv(2, 3, prevH)

        print("warping complete for set", imageSet)

        # WARPING COMPLETE. BLENDING START
        print("We are starting with blending")
        b = ImageBlenderWithPyramids()
        finalImg = cv2.imread(f'outputs/scene{imageSet}/opencv/warped_0.png')

        for index in range(1, 4):
            print('blending', index)
            img2 = cv2.imread(f'outputs/scene{imageSet}/opencv/warped_{index}.png')
            finalImg, mask1truth, mask2truth = b.blendImages(finalImg, img2)
            mask1truth = mask1truth + mask2truth
        
        cv2.imwrite(f'outputs/scene{imageSet}/opencv/blended_image.png', finalImg)
    else:
        prevH = np.eye(3)
        prevH = stitch_and_save_images_opencv(1, 0, prevH)
        prevH = stitch_and_save_images_opencv(1, 1, prevH)

        print("warping complete for set", imageSet)

        # WARPING COMPLETE. BLENDING START
        print("We are starting with blending")
        b = ImageBlenderWithPyramids()
        finalImg = cv2.imread(f'outputs/scene{imageSet}/opencv/warped_0.png')

        for index in range(1, 2):
            print('blending', index)
            img2 = cv2.imread(f'outputs/scene{imageSet}/opencv/warped_{index}.png')
            finalImg, mask1truth, mask2truth = b.blendImages(finalImg, img2)
            mask1truth = mask1truth + mask2truth
        
        cv2.imwrite(f'outputs/scene{imageSet}/opencv/blended_image.png', finalImg)


STARTING WITH OPENCV IMPLEMENTATION
STARTING WITH IMAGESET - 1
Original image size = (2448, 3264, 3)
Total number of matches: 1433
Original image size = (2448, 3264, 3)
Total number of matches: 1614
Original image size = (2448, 3264, 3)
Total number of matches: 1187
Original image size = (2448, 3264, 3)
Total number of matches: 1187
warping complete for set 1
We are starting with blending
blending 1
blending 2
blending 3
STARTING WITH IMAGESET - 2
Original image size = (1329, 2000, 3)
Total number of matches: 2133
Original image size = (1329, 2000, 3)
Total number of matches: 1633
Original image size = (1329, 2000, 3)
Total number of matches: 2626
Original image size = (1329, 2000, 3)
Total number of matches: 2792
warping complete for set 2
We are starting with blending
blending 1
blending 2
blending 3
STARTING WITH IMAGESET - 3
Original image size = (2448, 3264, 3)
Total number of matches: 1682
Original image size = (2448, 3264, 3)
Total number of matches: 1221
Original image size = (